In [1]:
!pip install recordlinkage --quiet
!pip install nameparser
!pip install pyphonetics
from pyphonetics import Soundex
import pandas as pd
import numpy as np
import recordlinkage
from nameparser import HumanName
from tqdm import tqdm
import re
from joblib import Parallel, delayed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.9 MB/s eta 0:00:00a 0:00:01


In [2]:
train_df_1 = pd.read_csv("/kaggle/input/linkage-hhhhack/main1.csv").drop_duplicates(subset='uid', keep=False)
# train_df_2 = pd.read_csv("/kaggle/input/linkage-hhhhack/main2.csv").drop_duplicates(subset='uid', keep=False)
# train_df_3 = pd.read_csv("/kaggle/input/linkage-hhhhack/main3.csv").drop_duplicates(subset='uid', keep=False)

In [3]:
def unique_words(name):
    if isinstance(name, str):
        words = name.split()
        return ' '.join(set(words))
    return ''

train_df_1 = train_df_1.dropna()

In [4]:
train_df_1['full_name'] = train_df_1['full_name'].apply(unique_words)

In [5]:
full_names = train_df_1['full_name'].tolist()
preproc_names = []

for i in tqdm(range(len(full_names))):
    preproc_names.append(HumanName(full_names[i]))

100%|██████████| 8624258/8624258 [15:30<00:00, 9266.28it/s]  


In [ ]:
def safe_phonetics(name):
    try:
        if isinstance(name, str) and name.strip():
            name = re.sub(r'5', '', name)
            name = re.sub(r'0', 'о', name)
            name = re.sub(r'-', '', name)
            name = re.sub(r'__', 'unk', name)
            name = re.sub(r'Ь+', '', name) 
            return soundex.phonetics(name)
        return ''
    except Exception as e:
        print(f"Ошибка при обработке имени: {name}. Ошибка: {e}")
        return 'UNK'  # Возвращаем пустую строку при ошибке

soundex = Soundex()
df_with_full_names = pd.DataFrame()

df_with_full_names['first_name'] = [name.first for name in tqdm(preproc_names)]
df_with_full_names['middle_name'] = [name.middle for name in tqdm(preproc_names)]
df_with_full_names['last_name'] = [name.last for name in tqdm(preproc_names)]

del preproc_names

df_with_full_names['middle_name_phnetic'] = df_with_full_names['middle_name'].apply(safe_phonetics)
df_with_full_names['first_name_phnetic'] = df_with_full_names['first_name'].apply(safe_phonetics)
df_with_full_names['last_name_phnetic'] = df_with_full_names['last_name'].apply(safe_phonetics)

100%|██████████| 8624258/8624258 [00:08<00:00, 1076563.34it/s]


Ошибка при обработке имени: -0. Ошибка: string index out of range
Ошибка при обработке имени: .. Ошибка: string index out of range
Ошибка при обработке имени: Ъ. Ошибка: string index out of range
Ошибка при обработке имени: '. Ошибка: string index out of range
Ошибка при обработке имени: -0. Ошибка: string index out of range
Ошибка при обработке имени: 0. Ошибка: string index out of range
Ошибка при обработке имени: 0. Ошибка: string index out of range
Ошибка при обработке имени: .. Ошибка: string index out of range
Ошибка при обработке имени: .. Ошибка: string index out of range
Ошибка при обработке имени: Ъ. Ошибка: string index out of range
Ошибка при обработке имени: .. Ошибка: string index out of range
Ошибка при обработке имени: _. Ошибка: string index out of range


In [ ]:
df_with_full_names

In [ ]:
# df_with_full_names['compare'] = df_with_full_names['first_name'].str[0]

In [ ]:
train_df_1